In [ ]:
!pip install mmcv-full==1.5.0 mmdet==2.24.1 mmpose==0.25.1 numpy==1.23.5 opencv-python-headless==4.8.0.74 openmim==0.1.5 timm==0.5.4 torch==1.11.0 torchvision==0.12.0

In [1]:
!pip install mmdet==2.24.1 mmpose==0.25.1 opencv-python-headless==4.8.0.74 openmim==0.1.5 timm==0.5.4 torch==1.11.0 torchvision==0.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.0/436.0 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 9.2 MB/s eta 0:00:00
Reason for being yanked: deprecated, use 4.8.0.76
  Created wheel for openmim: filename=openmim-0.1.5-py2.py3-none-any.whl size=42472 sha256=91e5b849631912a811c33cf76bcc8550a9e54994043ea6facb3

In [5]:
!pip install mmdet mmpose opencv-python-headless openmim timm torch torchvision

In [6]:
!mim install mmcv-full==1.5.0

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
mmcv-full==1.5.0 existed.


In [3]:
!git clone https://github.com/ViTAE-Transformer/ViTPose.git

Cloning into 'ViTPose'...
remote: Enumerating objects: 1859, done.
remote: Counting objects: 100% (1041/1041), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 1859 (delta 794), reused 725 (delta 725), pack-reused 818
Receiving objects: 100% (1859/1859), 10.75 MiB | 30.31 MiB/s, done.
Resolving deltas: 100% (958/958), done.


In [6]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd /content/mmdetection

Cloning into 'mmdetection'...
remote: Enumerating objects: 38019, done.
remote: Total 38019 (delta 0), reused 0 (delta 0), pack-reused 38019
Receiving objects: 100% (38019/38019), 63.25 MiB | 26.26 MiB/s, done.
Resolving deltas: 100% (26222/26222), done.
/content/mmdetection


In [11]:
from mmcv import Config

# 設定ファイルの読み込み
config_path = "/content/yolox_tiny_8x8_300e_coco.py"
cfg = Config.fromfile(config_path)

# `model`キーが存在するか確認
if 'model' in cfg:
    print("model key exists in the config.")
else:
    print("model key does not exist in the config.")


model key does not exist in the config.


In [3]:
!mim search mmdet --model "yolox"

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
--------------------------------------------------------------------------------
config id: yolox_s_8x8_300e_coco
architecture                                            cspdarknet,pafpn
coco/box_ap                                             40.5
config                                                  configs/yolox/yolox_s...
epochs                                                  300
model                                                   yolox
paper                                                   URL,Title
readme                                                  configs/yolox/README.md
training_data                                           coco
training_memory(GB)                                     7.6
training_resources                                      8x titanxp gpus
training_techniques                 

In [4]:
!mim download mmdet --config yolox_tiny_8x8_300e_coco --dest /content

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
processing yolox_tiny_8x8_300e_coco...
  [####################################]  100%          
Successfully downloaded yolox_tiny_8x8_300e_coco_20211124_171234-b4047906.pth to /content
Traceback (most recent call last):
  File "/usr/local/bin/mim", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1066, in invoke
    return ctx

In [8]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from huggingface_hub import hf_hub_download
from mmdet.apis import inference_detector, init_detector
from mmpose.apis import inference_top_down_pose_model, init_pose_model, process_mmdet_results, vis_pose_result
import urllib.request

# 物体検出モデルの初期化
class DetModel:
    MODEL_DICT = {
        "YOLOX-l": {
            "config": "https://github.com/open-mmlab/mmdetection/blob/master/configs/yolox/yolox_l_8x8_300e_coco.py",
            "model": "https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_l_8x8_300e_coco/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth",
        },
    }

    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model_name = "YOLOX-l"
        self.model = self._load_model(self.model_name)

    def _load_model(self, name: str):
        # d = self.MODEL_DICT[name]
        # config_url = d["config"]
        # config_path = "/content/yolox_l_8x8_300e_coco.py"
        # model_url = d["model"]
        # model_path = "/content/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth"
        # urllib.request.urlretrieve(config_url, config_path)
        # urllib.request.urlretrieve(model_url, model_path)
        config_path = "/content/yolox_tiny_8x8_300e_coco.py"
        model_path = "/content/yolox_tiny_8x8_300e_coco_20211124_171234-b4047906.pth"

        return init_detector(config_path, model_path)

    def detect(self, image: np.ndarray):
        image = image[:, :, ::-1]  # RGB -> BGR
        out = inference_detector(self.model, image)
        return out

    def visualize_detection_results(self, image: np.ndarray, detection_results: list, score_threshold: float = 0.3):
        person_det = [detection_results[0]] + [np.array([]).reshape(0, 5)] * 79
        image = image[:, :, ::-1]  # RGB -> BGR
        vis = self.model.show_result(
            image, person_det, score_thr=score_threshold, bbox_color=None, text_color=(200, 200, 200), mask_color=None
        )
        return vis[:, :, ::-1]  # BGR -> RGB



det_model = DetModel()

AttributeError: 'ConfigDict' object has no attribute 'model'

In [ ]:
# 姿勢推定モデルの初期化
class PoseModel:
    MODEL_DICT = {
        "ViTPose-B (single-task train)": {
            "config": "ViTPose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/ViTPose_base_coco_256x192.py",
            "model": "models/vitpose-b.pth",
        },
        "ViTPose-L (single-task train)": {
            "config": "ViTPose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/ViTPose_large_coco_256x192.py",
            "model": "models/vitpose-l.pth",
        },
        "ViTPose-B (multi-task train, COCO)": {
            "config": "ViTPose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/ViTPose_base_coco_256x192.py",
            "model": "models/vitpose-b-multi-coco.pth",
        },
        "ViTPose-L (multi-task train, COCO)": {
            "config": "ViTPose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/ViTPose_large_coco_256x192.py",
            "model": "models/vitpose-l-multi-coco.pth",
        },
    }

    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model_name = "ViTPose-B (single-task train)"
        self.model = self._load_model(self.model_name)

    def _load_model(self, name: str):
        d = self.MODEL_DICT[name]
        ckpt_path = hf_hub_download("public-data/ViTPose", d["model"])
        config_path = d["config"]
        model = init_pose_model(config_path, ckpt_path, device=self.device)
        return model

    def predict_pose(self, image: np.ndarray, det_results: list, box_score_threshold: float = 0.5):
        image = image[:, :, ::-1]  # RGB -> BGR
        person_results = process_mmdet_results(det_results, 1)
        out, _ = inference_top_down_pose_model(
            self.model, image, person_results=person_results, bbox_thr=box_score_threshold, format="xyxy"
        )
        return out

    def visualize_pose_results(self, image: np.ndarray, pose_results: list, kpt_score_threshold: float = 0.3, vis_dot_radius: int = 4, vis_line_thickness: int = 1):
        image = image[:, :, ::-1]  # RGB -> BGR
        vis = vis_pose_result(
            self.model,
            image,
            pose_results,
            kpt_score_thr=kpt_score_threshold,
            radius=vis_dot_radius,
            thickness=vis_line_thickness,
        )
        return vis[:, :, ::-1]  # BGR -> RGB

pose_model = PoseModel()

In [18]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from huggingface_hub import hf_hub_download
from mmdet.apis import inference_detector, init_detector
from mmpose.apis import inference_top_down_pose_model, init_pose_model, process_mmdet_results, vis_pose_result
import urllib.request

# # 物体検出モデルの初期化
# class DetModel:
#     MODEL_DICT = {
#         "YOLOX-l": {
#             "config": "https://github.com/open-mmlab/mmdetection/blob/master/configs/yolox/yolox_l_8x8_300e_coco.py",
#             "model": "https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_l_8x8_300e_coco/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth",
#         },
#     }

#     def __init__(self):
#         self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#         self.model_name = "YOLOX-l"
#         self.model = self._load_model(self.model_name)

#     def _load_model(self, name: str):
#         d = self.MODEL_DICT[name]
#         return init_detector(d["config"], d["model"], device=self.device)

#     def detect(self, image: np.ndarray):
#         image = image[:, :, ::-1]  # RGB -> BGR
#         out = inference_detector(self.model, image)
#         return out

#     def visualize_detection_results(self, image: np.ndarray, detection_results: list, score_threshold: float = 0.3):
#         person_det = [detection_results[0]] + [np.array([]).reshape(0, 5)] * 79
#         image = image[:, :, ::-1]  # RGB -> BGR
#         vis = self.model.show_result(
#             image, person_det, score_thr=score_threshold, bbox_color=None, text_color=(200, 200, 200), mask_color=None
#         )
#         return vis[:, :, ::-1]  # BGR -> RGB

# 姿勢推定モデルの初期化
class PoseModel:
    MODEL_DICT = {
        "ViTPose-B (single-task train)": {
            "config": "https://raw.githubusercontent.com/ViTAE-Transformer/ViTPose/main/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/ViTPose_base_coco_256x192.py",
            "model": "models/vitpose-b.pth",
        },
        "ViTPose-L (single-task train)": {
            "config": "ViTPose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/ViTPose_large_coco_256x192.py",
            "model": "models/vitpose-l.pth",
        },
        "ViTPose-B (multi-task train, COCO)": {
            "config": "ViTPose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/ViTPose_base_coco_256x192.py",
            "model": "models/vitpose-b-multi-coco.pth",
        },
        "ViTPose-L (multi-task train, COCO)": {
            "config": "ViTPose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/ViTPose_large_coco_256x192.py",
            "model": "models/vitpose-l-multi-coco.pth",
        },
    }

    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model_name = "ViTPose-B (single-task train)"
        self.model = self._load_model(self.model_name)

    def _download_base_files(self):
        base_files = {
            "_base_/default_runtime.py": "https://raw.githubusercontent.com/ViTAE-Transformer/ViTPose/main/configs/_base_/default_runtime.py",
            "_base_/datasets/coco.py": "https://raw.githubusercontent.com/ViTAE-Transformer/ViTPose/main/configs/_base_/datasets/coco.py"
        }
        for path, url in base_files.items():
            os.makedirs(os.path.dirname(path), exist_ok=True)
            urllib.request.urlretrieve(url, path)

    def _load_model(self, name: str):
        d = self.MODEL_DICT[name]
        ckpt_path = hf_hub_download("public-data/ViTPose", d["model"])
        config_url = d["config"]
        config_path = "/content/ViTPose_base_coco_256x192.py"

        # 必要なベースファイルをダウンロード
        self._download_base_files()

        urllib.request.urlretrieve(config_url, config_path)
        model = init_pose_model(config_path, ckpt_path, device=self.device)
        return model

    def predict_pose(self, image: np.ndarray, det_results: list, box_score_threshold: float = 0.5):
        image = image[:, :, ::-1]  # RGB -> BGR
        person_results = process_mmdet_results(det_results, 1)
        out, _ = inference_top_down_pose_model(
            self.model, image, person_results=person_results, bbox_thr=box_score_threshold, format="xyxy"
        )
        return out

    def visualize_pose_results(self, image: np.ndarray, pose_results: list, kpt_score_threshold: float = 0.3, vis_dot_radius: int = 4, vis_line_thickness: int = 1):
        image = image[:, :, ::-1]  # RGB -> BGR
        vis = vis_pose_result(
            self.model,
            image,
            pose_results,
            kpt_score_thr=kpt_score_threshold,
            radius=vis_dot_radius,
            thickness=vis_line_thickness,
        )
        return vis[:, :, ::-1]  # BGR -> RGB

# モデルの初期化
# det_model = DetModel()
pose_model = PoseModel()

FileNotFoundError: file "/_base_/default_runtime.py" does not exist

In [5]:
import torch

print(torch.__version__)

1.11.0+cu102


In [ ]:
# 画像のアップロード
uploaded = files.upload()

for filename in uploaded.keys():
    image_path = filename

# 画像の読み込み
image = plt.imread(image_path)

In [ ]:
# 物体検出
det_results = det_model.detect(image)
det_visualization = det_model.visualize_detection_results(image, det_results)

# 検出結果の表示
plt.figure(figsize=(10, 10))
plt.imshow(det_visualization)
plt.title("Detection Results")
plt.axis("off")
plt.show()

In [ ]:
# 姿勢推定
pose_results = pose_model.predict_pose(image, det_results)
pose_visualization = pose_model.visualize_pose_results(image, pose_results)

# 姿勢推定結果の表示
plt.figure(figsize=(10, 10))
plt.imshow(pose_visualization)
plt.title("Pose Estimation Results")
plt.axis("off")
plt.show()